# SDG 2.0 Demo

In [ ]:
# config
url = "http://127.0.0.1:8000"
username = "test-tay-5@sdg.com"
password = "pytest**pwd"

In [ ]:
import requests
import json
session = requests.session() 

In [ ]:
# resgister
register_response = session.post(url + "/auth/register", json={
    "email": username,
    "password": password,
    "is_active": True,
    "is_superuser": True,
    "is_verified": True
})

# login
login_response = session.post(url + "/auth/jwt/login", 
data={'username':username,'password':password},
headers={'Content-Type':'application/x-www-form-urlencoded'})
auth_value = '{} {}'.format(login_response.json()['token_type'], login_response.json()['access_token'])
print(auth_value)

In [ ]:
# current user info
response = session.get(url+"/users/me",
                      headers={'Authorization':auth_value})
rootfolder_id = response.json()['rootfolder']
print("#"*20 + " Current user info:")
print(json.dumps(response.json(), indent=2))

In [ ]:
# show folder info
def get_folder(folder_id):
    response = session.get(url+"/scen-folders/{}".format(folder_id),
                          headers={'Authorization':auth_value})
    return response
print("#"*20 + " Root folder of the current user:")
response = get_folder(rootfolder_id)
print(json.dumps(response.json(), indent=2))

In [ ]:
# create scenario
## read cartel script
with open('./jam.cartel') as f:
    script = f.read()
#     escaped_script = json.dumps(script)
## send post scenario request
data = {
          "script": script,
          "language": "cartel",
          "map_name": "Town03",
          "name": "follwing",
          "tags": [
            "test"
          ],
          "categories": [
            "SDGCAP"
          ]
        }
response = session.post(url + "/scen-folders/{}/scenarios".format(rootfolder_id), 
data=json.dumps(data),
headers={'Authorization':auth_value, 'Content-Type':'application/json'})
## show result
scenario1 = response.json()
scenario_id = scenario1["id"]
print("#"*20 + " The new scenario created in root folder:")
print(json.dumps(scenario1, indent=2))
print("#"*20 + " Check the root folder's scenarios_id list:")
response = get_folder(rootfolder_id)
print(json.dumps(response.json(), indent=2))

In [ ]:
# create task
def create_task(use_scenario_id,use_scenario, ads, time_limit):
    data = {
              "use_scenario_id": use_scenario_id,
              "use_scenario": use_scenario,
              "ads": ads,
              "time_limit": time_limit
            }
    response = session.post(url + "/tasks", 
                            data=json.dumps(data),
                            headers={'Authorization':auth_value, 'Content-Type':'application/json'})
    return response
## create a task based on the scenarios created above called "follwing"
response = create_task(scenario_id, None, "autoware", 100)
task1_id = response.json()["id"]
print("#"*20 + " The first task:")
task1 = response.json()
# task1["use_scenario"]["script"]="***Too long...... omitted"
print(json.dumps(task1, indent=2))

## create another task based on the same scanario
response = create_task(None, scenario1, "autoware", 80)
task2_id = response.json()["id"]
print("#"*20 + " The second task:")
task2 = response.json()
# task2["use_scenario"]["script"]="***Too long...... omitted"
print(json.dumps(task2, indent=2))

In [ ]:
# create job
def create_job(desc, tasks_id):
    data = {
              "desc": desc,
              "tasks_id": tasks_id,
            }
    response = session.post(url + "/jobs", 
                            data=json.dumps(data),
                            headers={'Authorization':auth_value, 'Content-Type':'application/json'})
    return response
response = create_job("Demo job with two tasks", [task1_id, task2_id])
job_id = response.json()['id']
print(json.dumps(response.json(), indent=2))

In [ ]:
# run job
def run_job(job_id):
    response = session.post(url + "/jobs/{}/run".format(job_id), 
                            headers={'Authorization':auth_value})
    return response
response = run_job(job_id)
print(json.dumps(response.json(), indent=2))

In [ ]:
# watch task status
def get_task(task_id):
    response = session.get(url+"/tasks/{}".format(task_id),
                          headers={'Authorization':auth_value})
    return response
task1 = get_task(task1_id).json()
print("The status of the first task:{}".format(task1['status']))
if task1['result']:
    print(task1['result'])
print("="*16)
task2 = get_task(task2_id).json()
print("The status of the second task:{}".format(task2['status']))
if task2['result']:
    print(task2['result'])